In [1]:
import json
import os
import time
import yaml
os.chdir('../')
from api.couch_db import TweetsDB

In [2]:
from glob import glob

In [3]:
import pandas as pd

In [4]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [5]:
#db connections
couch_db_conf = cfg['COUCHDB']
#create an instance of TWEETSDB and pass the connection details
couch_db = TweetsDB(couch_db_conf)

In [6]:
df = pd.read_csv('notebooks/SA2_2016_AUST.csv')

In [7]:
columns = ['SA4_CODE_2016', 'SA4_NAME_2016', 'GCCSA_NAME_2016', 'STATE_NAME_2016']

In [8]:
mappings_df = df[columns].drop_duplicates().set_index('SA4_CODE_2016')
mappings_df.head()

,SA4_NAME_2016,GCCSA_NAME_2016,STATE_NAME_2016
SA4_CODE_2016,,,
101,Capital Region,Rest of NSW,New South Wales
102,Central Coast,Greater Sydney,New South Wales
103,Central West,Rest of NSW,New South Wales
104,Coffs Harbour - Grafton,Rest of NSW,New South Wales
105,Far West and Orana,Rest of NSW,New South Wales


In [9]:
mappings_df.loc[101,['GCCSA_NAME_2016', 'STATE_NAME_2016']]

GCCSA_NAME_2016        Rest of NSW
STATE_NAME_2016    New South Wales
Name: 101, dtype: object

In [10]:
mappings_df.loc[101]['GCCSA_NAME_2016']

'Rest of NSW'

In [11]:
with open('notebooks/SA4_data_for_geotagging.json', 'r') as fp:
    data = json.load(fp)

In [12]:
data.keys()

dict_keys(['type', 'crs', 'features'])

In [13]:
features = data['features']

In [14]:
len(features)

89

In [15]:
features[0].keys()

dict_keys(['type', 'geometry', 'properties', 'id'])

In [16]:
features[4]['properties']

{'feature_code': '105',
 'feature_name': 'Far West and Orana',
 'median_tot_fam_inc_weekly': 1386,
 'median_tot_prsnl_inc_weekly': 590,
 'sa4_code16': '105',
 'sa4_name16': 'Far West and Orana'}

In [17]:
for i in range(len(features)):
    
    print(i,'\t',features[i]['properties']['feature_name'] ,'\t',len(features[i]['geometry']['coordinates']))


0 	 Capital Region 	 109
1 	 Central Coast 	 4
2 	 Central West 	 1
3 	 Coffs Harbour - Grafton 	 26
4 	 Far West and Orana 	 1
5 	 Hunter Valley exc Newcastle 	 42
6 	 Illawarra 	 7
7 	 Mid North Coast 	 40
8 	 Murray 	 1
9 	 New England and North West 	 1
10 	 Newcastle and Lake Macquarie 	 7
11 	 Richmond - Tweed 	 9
12 	 Riverina 	 1
13 	 Southern Highlands and Shoalhaven 	 4
14 	 Sydney - Baulkham Hills and Hawkesbury 	 2
15 	 Sydney - Blacktown 	 1
16 	 Sydney - City and Inner South 	 3
17 	 Sydney - Eastern Suburbs 	 5
18 	 Sydney - Inner South West 	 1
19 	 Sydney - Inner West 	 2
20 	 Sydney - North Sydney and Hornsby 	 3
21 	 Sydney - Northern Beaches 	 3
22 	 Sydney - Outer South West 	 1
23 	 Sydney - Outer West and Blue Mountains 	 1
24 	 Sydney - Parramatta 	 1
25 	 Sydney - Ryde 	 1
26 	 Sydney - South West 	 1
27 	 Sydney - Sutherland 	 1
28 	 Ballarat 	 1
29 	 Bendigo 	 1
30 	 Geelong 	 5
31 	 Hume 	 1
32 	 Latrobe - Gippsland 	 108
33 	 Melbourne - Inner 	 1
34 	 Melb

In [18]:
from shapely.geometry import Point, Polygon

In [19]:
a =[152.0326921, -28.19089878] 


In [20]:
def geo_check(coordinates):
    if isinstance(coordinates[0], list):              #check if polygon
        coordinate_polygon = Polygon(coordinates[0])
        pnt = Point(coordinate_polygon.centroid)

    else:
        pnt = Point(coordinates)
         
    for i in range(len(features)):
        for j in range(len(features[i]['geometry']['coordinates'])):
     
            poly = Polygon(features[i]['geometry']['coordinates'][j][0])
            idx = int(features[i]['properties']['sa4_code16'])
            
            if pnt.within(poly):
                return {'SA4_name':features[i]['properties']['feature_name'],
                        'SA4_code':idx,
                        'GCCSA_name':mappings_df.loc[idx]['GCCSA_NAME_2016'],
                        'State_name':mappings_df.loc[idx]['STATE_NAME_2016']}
                break
            
            elif pnt.touches(poly):
                return {'SA4_name':features[i]['properties']['feature_name'],
                        'SA4_code':idx,
                        'GCCSA_name':mappings_df.loc[idx]['GCCSA_NAME_2016'],
                        'State_name':mappings_df.loc[idx]['STATE_NAME_2016']}
                break
            

In [21]:
def none_geo_check(coordinates):
    dist_from_point = []
    dist_point_id = []
    if isinstance(coordinates[0], list):              #check if polygon
        coordinate_polygon = Polygon(coordinates[0])
        pnt = Point(coordinate_polygon.centroid)

    else:
        pnt = Point(coordinates)
         
    for i in range(len(features)):
        for j in range(len(features[i]['geometry']['coordinates'])):
     
            poly = Polygon(features[i]['geometry']['coordinates'][j][0])
            
            dist_from_point.append(poly.exterior.distance(pnt))
            dist_point_id.append(i)
            

    min_index = dist_point_id[dist_from_point.index(min(dist_from_point))]   
    idx = int(features[min_index]['properties']['sa4_code16'])
    return {'SA4_name':features[min_index]['properties']['feature_name'],
                        'SA4_code':idx,
                        'GCCSA_name':mappings_df.loc[idx]['GCCSA_NAME_2016'],
                        'State_name':mappings_df.loc[idx]['STATE_NAME_2016']}

        

In [22]:
print(geo_check([151.211, -33.86]))

None


In [23]:
geo_tasks_path= cfg['QUEUES']['geo_tasks']

In [ ]:
geo_tweets = glob('{}/*.txt'.format(geo_tasks_path))
for path in geo_tweets[:10]:
    tweet_id = path.split('/')[-1].split('.')[0]
    print(tweet_id)

991526112093028352
991493140270858240
991563363053223936
991531911758598144
991570085356568582
991556504162451456
991621232700215296
990046602940137472
991498889344643072
991554709021315072


In [ ]:
i = 1
while True:
    geo_tweets = glob('{}/*.txt'.format(geo_tasks_path))
 
    for path in geo_tweets:
        try:
            tweet_id = path.split('/')[-1].split('.')[0]
            with open(path, 'r') as fp:
                geo = json.load(fp)
            geo_doc =geo_check(geo['coordinates'])
            if geo_doc== None:
                none_geo = none_geo_check(geo['coordinates'])
                couch_db.update_document(tweet_id,none_geo)
            else:             
                couch_db.update_document(tweet_id,geo_doc)
            try:
                os.remove(path)
            except Exception as e:
                           # if failed, report it back to the user 
                print("Error: {} .".format(e))
        except Exception as e:
            print('Tweet {} wasn\'t geotagged and updated on DB due to error. {}'.format(tweet_id, e))
         
    print('Iteration: {}\tFiles processed: {}'.format(i, len(geo_tweets)))
    i+=1
    time.sleep(18)